# Data Warehouse Medicare National QA - Member Enrollment Monthly

Performing QA on member_enrollment_monthly table in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Table Information

This table contains enrollment information on a monthly level. Depending on the data source, this information can be easily extracted in a monthly level.

Data Sources:

* Optum Zip/Optum DoD: Enrollment information not on monthly level. Enrollment dates have a begin date and an end  date which may be longer than a month. Enrollment tables are **mbr_enroll** and **mbr_co_enroll**
* Truven: Enrollment table , **t**, contains monthly level enrollment data
* Medicaid: Enrollment tables (**enrl**, **chip_uth**, **htw_enrl**) are in month level usually identified by **elig_month/elig_date** column
* Medicare: Enrollment table (**mbsf_abcd_summary**) are in yearly level, to get monthly enrollment, you need to look at the **mdcr_status_code_** columns


Ideally we should have counts of enrollment tables from raw sources. These counts are included with the rest of the raw data tables counts for the given data sources.

* Optum Zip: **qa_reporting.optum_zip_counts**
* Optum Dod: **qa_reporting.optum_dod_counts**
* Medicaid: **qa_reporting.mdcd_enrollment_counts_[cy/fy]**
* Truven: **qa_reporting.truven_counts**
* Medicare: **qa_reporting.medicare_national_counts** and **qa_reporting.medicare_texas_counts**

## Row Counts and Enrollment Counts

In [3]:
query = ''' drop table if exists qa_reporting.dw_mcrn_mbr_enrl_monthly;
create table qa_reporting.dw_mcrn_mbr_enrl_monthly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    src_row_count int,
    row_count_diff int,
    row_count_diff_percentage float,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [4]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_mcrn_mbr_enrl_monthly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        current_date
  from dw_staging.mcrn_member_enrollment_monthly
 group by 1,2,3;
      '''

      cursor.execute(query)

      query = '''
update qa_reporting.dw_mcrn_mbr_enrl_monthly a
set src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.medicare_national_counts b
where data_source = 'mcrn'
and a.calendar_year = b.year
and a.table_src = 'medicare_national.' || b.table_name
;
      '''

      cursor.execute(query)


      query = '''
      with mcr_month_enrollment as (
            select year, bene_id, t.month_year_id
            from medicare_national.mbsf_abcd_summary a
            cross join lateral (values (a.year || '01', a.mdcr_status_code_01), (a.year || '02', a.mdcr_status_code_02),
                              (a.year || '03', a.mdcr_status_code_03), (a.year || '04', a.mdcr_status_code_04), (a.year || '05', a.mdcr_status_code_05),
                              (a.year || '06', a.mdcr_status_code_06), (a.year || '07', a.mdcr_status_code_07), (a.year || '08', a.mdcr_status_code_08),
                              (a.year || '09', a.mdcr_status_code_09), (a.year || '10', a.mdcr_status_code_10), (a.year || '11', a.mdcr_status_code_11),
                              (a.year || '12', a.mdcr_status_code_12))
            t(month_year_id, enrollment_status)
            where t.enrollment_status in ('10','11','20','21','31')
      ),
      mcr_month_enrl_count as (
            select year::int, count(*) as row_count
            from mcr_month_enrollment 
            group by 1
      )
      update qa_reporting.dw_mcrn_mbr_enrl_monthly a
      set src_row_count = b.row_count,
            row_count_diff = a.dw_row_count - b.row_count,
            row_count_diff_percentage = 100. * abs(a.dw_row_count - b.row_count) / b.row_count
      from mcr_month_enrl_count b
      where a.calendar_year = b.year
      '''
      
      cursor.execute(query)

After inserting the counts from the dw_staging schema, let's see if there are any years where the counts do not match with the raw tables.

In [5]:
query = '''
select * 
from qa_reporting.dw_mcrn_mbr_enrl_monthly
order by calendar_year
;'''
member_monthly_df = pd.read_sql(query, con=connection)
member_monthly_df


c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
0,mcrn,2014,medicare_national.mbsf_abcd_summary,32819005,32819005,0,0.0,2877140,2877140,2877408,-268,0.009314,2023-12-18
1,mcrn,2015,medicare_national.mbsf_abcd_summary,33818812,33818812,0,0.0,2962729,2962729,2963123,-394,0.013297,2023-12-18
2,mcrn,2016,medicare_national.mbsf_abcd_summary,34821230,34821230,0,0.0,3045820,3045820,3046466,-646,0.021205,2023-12-18
3,mcrn,2017,medicare_national.mbsf_abcd_summary,35827216,35827216,0,0.0,3132606,3132606,3133013,-407,0.012991,2023-12-18
4,mcrn,2018,medicare_national.mbsf_abcd_summary,36929927,36929927,0,0.0,3228969,3228969,3229338,-369,0.011426,2023-12-18
5,mcrn,2019,medicare_national.mbsf_abcd_summary,37904124,37904124,0,0.0,3306584,3306584,3306928,-344,0.010402,2023-12-18
6,mcrn,2020,medicare_national.mbsf_abcd_summary,38831588,38831588,0,0.0,3390643,3390643,3390959,-316,0.009319,2023-12-18
7,mcrn,2021,medicare_national.mbsf_abcd_summary,38201607,38201607,0,0.0,3248842,3248842,3248859,-17,0.000523,2023-12-18


In [6]:
member_monthly_df[(member_monthly_df['row_count_diff_percentage'] > 1.) | (member_monthly_df['mbr_count_percentage'] > 1.)]

,data_source,calendar_year,table_src,dw_row_count,src_row_count,row_count_diff,row_count_diff_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated


If **member_monthly_df** does not have any rows, it means that all of the rows from the raw tables are in this enrollment table at a monthly level.

## Gender Count

Now that we have verified that most if not all of the rows from the raw tables, ccaet and mdcrt, have been added to the member_enrollment_monthly table, we will check that the counts for other columns such as gender have been correctly added to the DW table.

In this case we won't seperate the counts by source table, just by calendar year.

In [7]:
query = '''with mcrn_gen_cd as (
    select year::int, bene_id, t.month_year_id, sex_ident_cd
            from medicare_national.mbsf_abcd_summary a
            cross join lateral (values (a.year || '01', a.mdcr_status_code_01), (a.year || '02', a.mdcr_status_code_02),
                              (a.year || '03', a.mdcr_status_code_03), (a.year || '04', a.mdcr_status_code_04), (a.year || '05', a.mdcr_status_code_05),
                              (a.year || '06', a.mdcr_status_code_06), (a.year || '07', a.mdcr_status_code_07), (a.year || '08', a.mdcr_status_code_08),
                              (a.year || '09', a.mdcr_status_code_09), (a.year || '10', a.mdcr_status_code_10), (a.year || '11', a.mdcr_status_code_11),
                              (a.year || '12', a.mdcr_status_code_12))
            t(month_year_id, enrollment_status)
            where t.enrollment_status in ('10','11','20','21','31')
),
mcrn_gen as (
    select year, c.gender_cd, count(*) gender_count
    from mcrn_gen_cd m
    left join reference_tables.ref_gender c
    on c.data_source = 'mcr'
   and c.gender_cd_src = m.sex_ident_cd
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.mcrn_member_enrollment_monthly
    group by 1,2
)
select a.year, a.gender_cd, a.gender_count as dw_gender_count, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_count_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_count_diff_percentage
from mcrn_gen b
full outer join dw_gen a
on a.year = b.year
and a.gender_cd = b.gender_cd;
'''
 
df = pd.read_sql(query,  con=connection)
df.sort_values(['year', 'gender_cd'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,gender_cd,dw_gender_count,src_gender_count,gender_count_diff,gender_count_diff_percentage
3,2014,F,17987802,17987802,0,0.0
10,2014,M,14831203,14831203,0,0.0
2,2015,F,18532633,18532633,0,0.0
0,2015,M,15286179,15286179,0,0.0
14,2016,F,19085536,19085536,0,0.0
12,2016,M,15735694,15735694,0,0.0
5,2017,F,19639731,19639731,0,0.0
4,2017,M,16187485,16187485,0,0.0
8,2018,F,20265227,20265227,0,0.0
11,2018,M,16664700,16664700,0,0.0


## Plan Type Counts

In [8]:
# Including enrollments where the plantyp column is NULL. Treating it as if unknown.
query = '''with mcrn_enroll as (
    select year::int, bene_id, ent.plan_type
    from medicare_national.mbsf_abcd_summary a
    cross join lateral (values (01, a.mdcr_entlmt_buyin_ind_01, a.mdcr_status_code_01), (02, a.mdcr_entlmt_buyin_ind_02, a.mdcr_status_code_02),
                        (03, a.mdcr_entlmt_buyin_ind_03, a.mdcr_status_code_03), (04, a.mdcr_entlmt_buyin_ind_04, a.mdcr_status_code_04), (05, a.mdcr_entlmt_buyin_ind_05, a.mdcr_status_code_05),
                        (06, a.mdcr_entlmt_buyin_ind_06, a.mdcr_status_code_06), (07, a.mdcr_entlmt_buyin_ind_07, a.mdcr_status_code_07), (08, a.mdcr_entlmt_buyin_ind_08, a.mdcr_status_code_08),
                        (09, a.mdcr_entlmt_buyin_ind_09, a.mdcr_status_code_09), (10, a.mdcr_entlmt_buyin_ind_10, a.mdcr_status_code_10), (11, a.mdcr_entlmt_buyin_ind_11, a.mdcr_status_code_11),
                        (12, a.mdcr_entlmt_buyin_ind_12, a.mdcr_status_code_12))
    t(month_year_id, mcdcr_enrlmt, enrollment_status)
    join reference_tables.ref_medicare_entlmt_buyin ent 
    on ent.buyin_cd = t.mcdcr_enrlmt
    where t.enrollment_status in ('10','11','20','21','31')
  ),
mcrn_plans as (          
    select year, case when plan_type is null then 'UNK' else plan_type end as plan_type, count(*) plan_count
    from mcrn_enroll a
    group by 1,2
),
dw_plans as (
    select year, case when plan_type is null then 'UNK' else plan_type end as plan_type,
            count(*) plan_count
    from dw_staging.mcrn_member_enrollment_monthly
    group by 1,2
)
select a.year, a.plan_type, a.plan_count as dw_plan_count, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_count_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_count_diff_percentage
from mcrn_plans b
full outer join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_count_df = pd.read_sql(query,  con=connection)
plan_count_df.sort_values(['year', 'plan_type'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
0,2014,A,2860395,2860661.0,-266.0,0.009299
3,2014,AB,19945280,29752648.0,-9807368.0,32.963009
1,2014,B,190686,205696.0,-15010.0,7.297176
2,2014,C,9822644,NaN,NaN,NaN
6,2015,A,2958810,2959069.0,-259.0,0.008753
5,2015,AB,20061131,30652707.0,-10591576.0,34.553477
7,2015,B,191952,207036.0,-15084.0,7.285689
4,2015,C,10606919,NaN,NaN,NaN
9,2016,A,3072595,3073016.0,-421.0,0.013700
8,2016,AB,20376129,31540322.0,-11164193.0,35.396573


In [9]:
plan_count_df[plan_count_df['plan_count_diff_percentage'] > 1.0]

,year,plan_type,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
1,2014,B,190686,205696.0,-15010.0,7.297176
3,2014,AB,19945280,29752648.0,-9807368.0,32.963009
5,2015,AB,20061131,30652707.0,-10591576.0,34.553477
7,2015,B,191952,207036.0,-15084.0,7.285689
8,2016,AB,20376129,31540322.0,-11164193.0,35.396573
11,2016,B,196476,207892.0,-11416.0,5.491313
12,2017,B,192822,204806.0,-11984.0,5.851391
14,2017,AB,20354206,32419383.0,-12065177.0,37.215937
16,2018,AB,20332949,33379879.0,-13046930.0,39.086211
19,2018,B,192401,208234.0,-15833.0,7.603465


In [10]:
yearly_plan_count_df = plan_count_df.groupby('year')['dw_plan_count', 'src_plan_count'].sum()
yearly_plan_count_df['plan_count_diff'] = yearly_plan_count_df['dw_plan_count'] - yearly_plan_count_df['src_plan_count']
yearly_plan_count_df['plan_count_diff_percentage'] = 100.* abs(yearly_plan_count_df['plan_count_diff'] / yearly_plan_count_df['src_plan_count'])
yearly_plan_count_df

C:\Users\iperez9\AppData\Local\Temp\4\ipykernel_20128\850223352.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  yearly_plan_count_df = plan_count_df.groupby('year')['dw_plan_count', 'src_plan_count'].sum()


,dw_plan_count,src_plan_count,plan_count_diff,plan_count_diff_percentage
year,,,,
2014,32819005,32819005.0,0.0,0.0
2015,33818812,33818812.0,0.0,0.0
2016,34821230,34821230.0,0.0,0.0
2017,35827216,35827216.0,0.0,0.0
2018,36929927,36929927.0,0.0,0.0
2019,37904124,37904124.0,0.0,0.0
2020,38831588,38831588.0,0.0,0.0
2021,38201607,38201607.0,0.0,0.0
